In [17]:
import pandas as pd
import numpy as np
import numba as nb
import math

df = pd.read_csv(r'C:\Users\David\Documents\code\M1-13-PROJECT1\bank_data.csv')

In [13]:
@nb.njit(fastmath=True)
def trav_time(x1,y1, x2,y2, speed = 30):
    return np.sqrt((x1-x2)**2 + (y1-y2) **2 ) / speed

@nb.njit(fastmath=True)
def create_trav_time_mat(x_coo, y_coo):
    time_mat = np.zeros((len(y_coo), len(x_coo)))
    for i in range(len(x_coo)):
        for j in range(len(y_coo)):
            time_mat[i,j] = trav_time(x_coo[i], y_coo[i], x_coo[j], y_coo[j])
    return time_mat

x_coo = df['x_coordinate'].to_numpy()
y_coo = df['y_coordinate'].to_numpy()
# create a travel time matrix and use it later to get the travel time between each two banks
travel_time_mat= create_trav_time_mat(x_coo, y_coo)

In [106]:
def short_list(now_node, df, dont_list, home_flag):

    counter = 300
    banks = []
    indexer = 0

    bank_hold = 0
    dist_hold = 0

    if home_flag == False:
        while counter >= 0:
            if  df.iloc[indexer]['id'] in dont_list:
                indexer += 1
            else:
                banks.append( int( df.iloc[indexer]['id'] ) )
                indexer += 1
                counter -= 1
    
        for bank in banks:

            if travel_time_mat[ int( now_node.id ), bank] > dist_hold:
                bank_hold = bank
                dist_hold = travel_time_mat[ int( now_node.id ), bank]
            else:
                pass

    if home_flag == True:
        df.sort_values(by= ['home_run'], inplace = True)

        while counter >= 0:
            if  df.iloc[indexer]['id'] in dont_list:
                indexer += 1
            else:
                banks.append( int( df.iloc[indexer]['id'] ) )
                indexer += 1
                counter -= 1
    
        for bank in banks:

            if travel_time_mat[ int( now_node.id ), bank] > dist_hold:
                bank_hold = bank
                dist_hold = travel_time_mat[ int( now_node.id ), bank]
            else:
                pass

    return bank_hold, dist_hold

In [113]:
def robber_algorithm(df):

    #Initiate a few lists for later.
    payout = 0
    path = []
    dont_list = []
    time = 24

    home_flag = False

    #Create a new column with a ratio of money against time.
    df['cash_time'] = ( df['money'] / df['time (hr)'] )
    df['home_run'] = (np.sqrt((0-df['x_coordinate'])**2 + (0-df['y_coordinate'])**2) / 30)
    df.sort_values(by= ['cash_time'], inplace = True, ascending=False)

    #The path has to begin with a starting point. In this case, I chose the most expensive bank.
    now_node = df.loc[df['money'] == max(df['money'])].iloc[0]

    #Method
    time_home = (np.sqrt((0-df['x_coordinate'][now_node.id])**2 + (0-df['y_coordinate'][now_node.id])**2) / 30)
    
    while (time - time_home) > 0.5:
        if time < 3:
            home_flag = True
        path.append( int(now_node.id) )
        dont_list.append( int(now_node.id) )    
        next_node, next_dist = short_list(now_node, df, dont_list, home_flag)
        time -= df.loc[df.id == now_node.id, 'time (hr)'].iloc[0]
        time -=  next_dist
        payout += df.loc[df.id == now_node.id, 'money'].iloc[0]
        now_node = df.loc[df.id == next_node].iloc[0]

    else:
        if time < 0:
            woops = path.pop()
            time += next_dist
            payout -= df.loc[df.id == woops, 'money'].iloc[0]

            now_node = df.loc[df.id == path[-1]].iloc[0]
            time -= time_home
            
            print('You stole $', payout, 'with ', round(time, 2),'hours remaining. Your path was:', path)

        else:
            time -= time_home
            print('You stole $', payout, 'with ', round(time, 2),'hours remaining. Your path was:', path)

robber_algorithm(df)

You stole $ 2415100 with  0.2 hours remaining. Your path was: [4848, 6561, 6623, 7625, 299, 317, 4762, 2346, 4983, 2741, 6468, 3526, 5295, 9081, 1914, 1397, 8373, 8605, 8286, 8909, 1058, 5518, 4742, 9241, 2037, 2928, 6712, 6156, 8579, 9883, 8784, 8469, 2521, 4465, 3502, 5166, 4175, 7665, 3917, 8846, 9908, 9640, 3136, 8231, 1193, 2626, 576, 6585]
